In [1]:
import pandas as pd
import pickle
import re
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from scipy.optimize import linear_sum_assignment
import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
x = pickle.load(open("../data/sim_result_with_firm_dtm.pkl",'rb'))

In [3]:
scores = pd.DataFrame([item[0:3]+item[3]+[len(item[4]), len(item[5])] for item in x], \
                      columns = ["cik",'y1','y2', 's1', 's2', 's3', 's4', 'l1','l2'])

scores.head()

,cik,y1,y2,s1,s2,s3,s4,l1,l2
0,2135,2001,2002,0.752746,0.752746,0.769208,0.769208,34,91
1,2135,2001,2003,0.544492,0.544492,0.566316,0.566316,34,107
2,2135,2001,2004,0.484842,0.484842,0.513163,0.513163,34,128
3,2135,2002,2003,0.689329,0.689329,0.704758,0.704758,91,107
4,2135,2002,2004,0.621834,0.621834,0.646809,0.646809,91,128


In [4]:
scores["window"]= scores.y2-scores.y1
len(scores)
scores.head()

11519

,cik,y1,y2,s1,s2,s3,s4,l1,l2,window
0,2135,2001,2002,0.752746,0.752746,0.769208,0.769208,34,91,1
1,2135,2001,2003,0.544492,0.544492,0.566316,0.566316,34,107,2
2,2135,2001,2004,0.484842,0.484842,0.513163,0.513163,34,128,3
3,2135,2002,2003,0.689329,0.689329,0.704758,0.704758,91,107,1
4,2135,2002,2004,0.621834,0.621834,0.646809,0.646809,91,128,2


In [5]:
scores.window.value_counts()

1    4305
2    3825
3    3389
Name: window, dtype: int64

In [30]:
gvk_cik = pd.read_csv('/home/jujun/temp_10k/fscore_data/wciklink_gvkey_20200109.csv')

In [31]:
gvk_cik.head()

,gvkey,conm,DATADATE1,DATADATE2,cik,SOURCE,coname,FNDATE,LNDATE,N10K,N10K_A,FLAG
0,1000,A & E PLASTIK PAK INC,31-Dec-64,31-Dec-77,NaN,NOLNK,NaN,NaN,NaN,NaN,NaN,0
1,1001,A & M FOOD SERVICES INC,31-Dec-78,31-Dec-85,723576.0,COMPN,NaN,NaN,NaN,NaN,NaN,1
2,1002,AAI CORP,31-Dec-68,31-Dec-72,1745.0,COMPN,NaN,NaN,NaN,NaN,NaN,1
3,1002,AAI CORP,31-Dec-68,31-Dec-72,1306124.0,FDATE,AAI CORP,12-Nov-04,11-May-05,0.0,0.0,3
4,1003,A.A. IMPORTING CO INC,31-Dec-80,31-Jan-90,730052.0,COMPN,NaN,NaN,NaN,NaN,NaN,1


In [32]:
gvk_cik.gvkey.isnull().sum()

0

In [33]:
gvk_cik.cik.isnull().sum()

13832

In [34]:
gvk_cik_1 = gvk_cik.dropna()

In [36]:
gvk_cik_1.cik.isnull().sum()

0

In [37]:
def check(cik_list,x):
    r = 0
    if x in cik_list:
        r = 1
    return r

In [41]:
cik_list = list(set(scores.cik))
len(cik_list)

489

In [39]:
gvk_cik_1['keep_cik'] = gvk_cik_1.cik.apply(lambda x: check(cik_list,x))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
gvk_cik_1.keep_cik.value_counts()

0    26128
1      497
Name: keep_cik, dtype: int64

In [42]:
gvk_cik_fl = gvk_cik_1[gvk_cik_1.keep_cik == 1]

In [43]:
gvk_cik_fl.shape

(497, 13)

In [69]:
from datetime import datetime

In [76]:
def convert_date(x):
    date_time_obj = datetime. strptime(x, '%d-%b-%y')
    return date_time_obj.year

In [77]:
gvk_cik_fl['date1'] = gvk_cik_fl.DATADATE1.apply(lambda x: convert_date(x))
gvk_cik_fl['date2'] = gvk_cik_fl.DATADATE2.apply(lambda x: convert_date(x))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [79]:
gvk_cik_fl.head()

,gvkey,conm,DATADATE1,DATADATE2,cik,SOURCE,coname,FNDATE,LNDATE,N10K,N10K_A,FLAG,keep_cik,date1,date2
178,1164,MCI INC,31-Dec-74,31-Dec-04,723527.0,COMPN,MCI INC,21-Nov-94,27-Sep-06,13.0,3.0,3,1,1974,2004
188,1173,AEROSONIC CORP,31-Jan-74,31-Jan-13,109471.0,COMPN,AEROSONIC CORP /DE/,26-Apr-95,8-Jul-13,26.0,4.0,3,1,1974,2013
307,1278,SUNBEAM CORPORATION,31-Dec-50,31-Dec-99,3662.0,COMPN,SUNBEAM CORP/FL/,2-Feb-96,30-Dec-02,10.0,3.0,3,1,2050,1999
483,1447,AMERICAN EXPRESS CO,31-Dec-60,31-Dec-17,4962.0,COMPN,AMERICAN EXPRESS CO,25-Jan-94,16-Sep-19,29.0,4.0,3,1,2060,2017
527,1487,AMERICAN INTERNATIONAL GROUP,31-Dec-72,31-Dec-17,5272.0,COMPN,AMERICAN INTERNATIONAL GROUP INC,30-Mar-94,23-Aug-19,38.0,12.0,3,1,1972,2017


In [51]:
gvk_cik_fl[gvk_cik_fl.duplicated(subset=['gvkey','cik'],keep=False)]

,gvkey,conm,DATADATE1,DATADATE2,cik,SOURCE,coname,FNDATE,LNDATE,N10K,N10K_A,FLAG,keep_cik


In [49]:
gvk_cik_fl[gvk_cik_fl.duplicated(subset=['gvkey'],keep=False)]

,gvkey,conm,DATADATE1,DATADATE2,cik,SOURCE,coname,FNDATE,LNDATE,N10K,N10K_A,FLAG,keep_cik
12343,12485,ALTICE USA INC,31-Dec-85,31-Dec-16,784681.0,FDATE,CSC HOLDINGS LLC,10-Feb-94,6-Jul-16,43.0,18.0,2,1
12344,12485,ALTICE USA INC,31-Dec-85,31-Dec-16,1053112.0,CUSIP,CABLEVISION SYSTEMS CORP /NY,20-Jan-98,6-Jul-16,32.0,13.0,2,1


In [52]:
gvk_cik_fl_dplc = gvk_cik_fl[gvk_cik_fl.duplicated(subset=['cik'],keep=False)]

In [53]:
gvk_cik_fl_dplc.shape

(50, 13)

In [54]:
dplc_cik = list(set(gvk_cik_fl_dplc.cik))

In [140]:
def match_cik_gvk(c,y, dplc_cik,gvk_cik_fl):
    gvk = None
    if c in dplc_cik:
        temp_gvk_cik = gvk_cik_fl[gvk_cik_fl.cik == c]
        for i, idx in enumerate(temp_gvk_cik.index):
            if temp_gvk_cik.date1[idx] <= y <= temp_gvk_cik.date2[idx]:
                gvk = temp_gvk_cik.gvkey[idx]
        if gvk == None:
            temp_gvk_cik = temp_gvk_cik.sort_values(by = ['date2'])
            if len(temp_gvk_cik) == 2:
                if y <= temp_gvk_cik.date2.iloc[0]:
                    gvk = temp_gvk_cik.gvkey.iloc[0]
                else:
                    gvk = temp_gvk_cik.gvkey.iloc[1]
            
    else:
        temp_gvk_cik = gvk_cik_fl[gvk_cik_fl.cik == c]
        if len(temp_gvk_cik) >=1:
            gvk = temp_gvk_cik.gvkey.iloc[0]
    return gvk

In [141]:
scores.cik.isnull().sum()

0

In [142]:
scores['gvkey'] = None

In [143]:
scores['gvkey'] = scores.apply(lambda x: match_cik_gvk(x.cik, x.y1, dplc_cik, gvk_cik_fl),axis=1)

In [164]:
temp = scores[scores.cik == 833444]
for idx in temp.index:
    if np.isnan(temp.gvkey[idx]):
        scores.loc[idx,'gvkey'] = 6268

In [165]:
scores[scores.cik == 833444]

,cik,y1,y2,s1,s2,s3,s4,l1,l2,window,gvkey
5125,833444,1996,1998,0.313915,0.179380,0.311784,0.176618,63,36,2,15349.0
5126,833444,1996,1999,0.286647,0.286647,0.285140,0.285140,63,190,3,15349.0
5127,833444,1998,1999,0.651483,0.651483,0.653042,0.653042,36,190,1,6268.0
5128,833444,1998,2000,0.322836,0.170386,0.372121,0.184419,36,19,2,6268.0
5129,833444,1998,2001,0.197315,0.186353,0.203955,0.197506,36,34,3,6268.0
5130,833444,1999,2000,0.844624,0.084462,0.818692,0.093798,190,19,1,6268.0
5131,833444,1999,2001,0.470546,0.084203,0.485261,0.099173,190,34,2,6268.0
5132,833444,1999,2002,0.603598,0.073067,0.568589,0.088717,190,23,3,6268.0
5133,833444,2000,2001,0.684369,0.684369,0.709553,0.709553,19,34,1,6268.0
5134,833444,2000,2002,0.562977,0.562977,0.595665,0.595665,19,23,2,6268.0


In [145]:
scores.gvkey.isnull().sum()

303

In [166]:
scores[scores.gvkey.isnull()].groupby('cik').size()

cik
27960      21
76348       9
99771      30
921066      5
1021009     6
1029315     3
1062982     3
1095722     3
1108955    12
1157522    24
1220606    27
1277151    21
1312910    27
1329011    15
1343504     6
1377848    24
1387361    12
1497814     1
dtype: int64

In [170]:
gvk_cik_fl[gvk_cik_fl.cik == 99771]

,gvkey,conm,DATADATE1,DATADATE2,cik,SOURCE,coname,FNDATE,LNDATE,N10K,N10K_A,FLAG,keep_cik,date1,date2


In [58]:
gvk_cik_fl[gvk_cik_fl.duplicated(subset=['cik','FLAG'],keep=False)]

,gvkey,conm,DATADATE1,DATADATE2,cik,SOURCE,coname,FNDATE,LNDATE,N10K,N10K_A,FLAG,keep_cik
3346,4093,DUKE ENERGY CORP,31-Dec-51,31-Dec-17,1326160.0,COMPN,DUKE ENERGY CORP,9-May-05,12-Sep-19,16.0,3.0,3,1
6067,6598,PHOTOMEDEX INC-OLD,31-Dec-82,31-Dec-10,711665.0,CUSIP,"GADSDEN PROPERTIES, INC.",19-Jul-96,20-Sep-19,39.0,10.0,2,1
8895,9224,SAFETY-KLEEN CORP,30-Sep-82,31-Aug-99,701856.0,COMPN,SAFETY KLEEN CORP/,25-Jan-94,29-Dec-03,12.0,3.0,3,1
9040,9357,SAFETY-KLEEN CORP-OLD,31-Dec-79,31-Dec-97,701856.0,CUSIP,SAFETY KLEEN CORP/,25-Jan-94,29-Dec-03,12.0,3.0,3,1
10565,10787,TYCO INTERNATIONAL PLC,31-May-64,30-Sep-15,833444.0,FDATE,JOHNSON CONTROLS INTERNATIONAL PLC,9-Feb-95,13-Sep-19,32.0,8.0,2,1
11271,11450,WESTWOOD ONE INC -OLD,30-Nov-83,31-Dec-10,771950.0,CUSIP,"DIAL GLOBAL, INC. /DE/",11-Jan-94,12-Apr-13,29.0,7.0,2,1
13443,13510,DUKE REALTY CORP,31-Dec-86,31-Dec-17,1326160.0,CUSIP,DUKE ENERGY CORP,9-May-05,12-Sep-19,16.0,3.0,3,1
14470,14541,SURETY CAPITAL CORP -OLD,31-Oct-86,31-Oct-89,784932.0,CUSIP,SURETY CAPITAL CORP /DE/,13-Feb-95,23-May-08,21.0,1.0,3,1
15294,15349,ADT LTD,31-Dec-85,31-Dec-96,833444.0,FDATE,JOHNSON CONTROLS INTERNATIONAL PLC,9-Feb-95,13-Sep-19,32.0,8.0,2,1
17875,19537,STERLING FINANCIAL CORP,31-Dec-93,31-Dec-06,811671.0,COMPN,STERLING FINANCIAL CORP /PA/,9-Feb-94,8-Apr-08,17.0,3.0,3,1


In [332]:
scores = pd.read_csv('scores_gvk_20200620.csv')

In [334]:
scores = scores.drop(['Unnamed: 0','year'],axis=1)
scores.head()

,cik,y1,y2,s1,s2,s3,s4,l1,l2,window,gvkey,label,label_lag
0,2135,2001,2002,0.752746,0.752746,0.769208,0.769208,34,91,1,30697.0,1,1
1,2135,2001,2003,0.544492,0.544492,0.566316,0.566316,34,107,2,30697.0,1,1
2,2135,2001,2004,0.484842,0.484842,0.513163,0.513163,34,128,3,30697.0,1,1
3,2135,2002,2003,0.689329,0.689329,0.704758,0.704758,91,107,1,30697.0,1,1
4,2135,2002,2004,0.621834,0.621834,0.646809,0.646809,91,128,2,30697.0,1,1


In [193]:
with open("fraudsdata_20200525_2012", 'rb') as fb:
    data = pickle.load(fb)

In [194]:
data = pd.DataFrame(data, columns =['text','label','cik','year', 'file','label_lag'])
data.head()

,text,label,cik,year,file,label_lag
0,[quantitative and qualitative disclosures abou...,0,1122304,2015,edgar/data/1122304/0001122304-16-000201.txt,0
1,[item dd . managements discussion and analysis...,0,1122304,2002,edgar/data/1122304/0000950123-03-002268.txt,0
2,[item dd . managements discussion and analysis...,0,1122304,2003,edgar/data/1122304/0000950123-04-002526.txt,0
3,[quantitative and qualitative disclosures abou...,0,1122304,2014,edgar/data/1122304/0001122304-15-000038.txt,0
4,[quantitative and qualitative disclosures abou...,0,1122304,2011,edgar/data/1122304/0001122304-12-000045.txt,0


In [301]:
data["sen_num"] = data["text"].apply(lambda x: len(x))
data["word_num"] = data["text"].apply(lambda x: len(re.findall(r"\w+", ' '.join(x))))

In [311]:
data.head()
data.shape

,text,label,cik,year,file,label_lag,sen_num,word_num
0,[quantitative and qualitative disclosures abou...,0,1122304,2015,edgar/data/1122304/0001122304-16-000201.txt,0,5,79
1,[item dd . managements discussion and analysis...,0,1122304,2002,edgar/data/1122304/0000950123-03-002268.txt,0,8,98
2,[item dd . managements discussion and analysis...,0,1122304,2003,edgar/data/1122304/0000950123-04-002526.txt,0,8,104
3,[quantitative and qualitative disclosures abou...,0,1122304,2014,edgar/data/1122304/0001122304-15-000038.txt,0,5,79
4,[quantitative and qualitative disclosures abou...,0,1122304,2011,edgar/data/1122304/0001122304-12-000045.txt,0,5,78


(6273, 8)

In [335]:
temp_data = data[['cik','year', 'sen_num','word_num']]
scores = scores.merge(temp_data, \
                      how='left', left_on=['cik', 'y2'], right_on=['cik','year'])

In [336]:
scores.head()

,cik,y1,y2,s1,s2,s3,s4,l1,l2,window,gvkey,label,label_lag,year,sen_num,word_num
0,2135,2001,2002,0.752746,0.752746,0.769208,0.769208,34,91,1,30697.0,1,1,2002,113,8064
1,2135,2001,2003,0.544492,0.544492,0.566316,0.566316,34,107,2,30697.0,1,1,2003,134,10260
2,2135,2001,2004,0.484842,0.484842,0.513163,0.513163,34,128,3,30697.0,1,1,2004,152,13213
3,2135,2002,2003,0.689329,0.689329,0.704758,0.704758,91,107,1,30697.0,1,1,2003,134,10260
4,2135,2002,2004,0.621834,0.621834,0.646809,0.646809,91,128,2,30697.0,1,1,2004,152,13213


In [337]:
scores.groupby(["cik","y2"]).size().sort_values(ascending=False)

cik      y2  
784932   2003    27
1066107  2005    12
1007800  2001    12
911149   1999    12
806514   2003    12
803014   1997     8
793971   1997     8
73887    2008     8
23082    2008     8
785968   2007     8
894651   1995     8
833444   1999     8
1019376  1997     4
1060822  2003     4
866535   1997     4
3116     1996     4
832813   1998     4
826154   1996     4
912894   2000     3
878522   2010     3
         2011     3
         2012     3
878556   1999     3
84112    2008     3
878522   2007     3
         2009     3
         2008     3
878556   2001     3
878522   2006     3
         2005     3
                 ..
60714    1994     1
914565   1997     1
915840   2003     1
59870    1996     1
915840   2016     1
916802   1996     1
911115   1997     1
910406   1997     1
895516   1997     1
910329   1997     1
895516   2010     1
895642   1997     1
895651   1997     1
896880   1997     1
897074   1996     1
898437   1999     1
71829    1996     1
899723   1997     1
70578 

In [338]:
scores.isnull().sum()

cik            0
y1             0
y2             0
s1             0
s2             0
s3             0
s4             0
l1             0
l2             0
window         0
gvkey        249
label          0
label_lag      0
year           0
sen_num        0
word_num       0
dtype: int64

In [197]:
# scores.to_csv('scores_gvk_20200620.csv')

In [339]:
scores_1 = scores.dropna(subset= ['gvkey'])
len(scores)
len(scores_1)

11639

11390

In [340]:
map_key = scores_1.gvkey.apply(lambda x: str(int(x))) + "_" + scores_1.y1.apply(lambda x: str(int(x)))
scores_1['map_key'] = map_key

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [341]:
scores_1.head()

,cik,y1,y2,s1,s2,s3,s4,l1,l2,window,gvkey,label,label_lag,year,sen_num,word_num,map_key
0,2135,2001,2002,0.752746,0.752746,0.769208,0.769208,34,91,1,30697.0,1,1,2002,113,8064,30697_2001
1,2135,2001,2003,0.544492,0.544492,0.566316,0.566316,34,107,2,30697.0,1,1,2003,134,10260,30697_2001
2,2135,2001,2004,0.484842,0.484842,0.513163,0.513163,34,128,3,30697.0,1,1,2004,152,13213,30697_2001
3,2135,2002,2003,0.689329,0.689329,0.704758,0.704758,91,107,1,30697.0,1,1,2003,134,10260,30697_2002
4,2135,2002,2004,0.621834,0.621834,0.646809,0.646809,91,128,2,30697.0,1,1,2004,152,13213,30697_2002


In [20]:
f1_tb = pd.read_csv('/home/jujun/temp_10k/compf_0920_fl.csv')

In [22]:
f1_tb = f1_tb.drop(['Unnamed: 0'],axis=1)

In [23]:
f1_tb.head()

,gvkey,map_key,SIC,datadate,fyear,conm,che,rect,invt,act,...,nco,fin,soft_asset,rsst_acc,ch_rec,ch_inv,cs,ch_cs,issue,ch_roa
0,100004,100004_2000,2800,14975.0,2000.0,ALTANA AG,458.549,298.131,235.884,1053.789,...,334.415,211.1869,0.460417,-0.522628,-0.102745,-0.054366,1987.9749,-0.223429,0,0.015125
1,100004,100004_2001,2800,15340.0,2001.0,ALTANA AG,324.609,358.393,243.299,992.267,...,408.872,171.9379,0.556220,0.036269,0.033727,0.004150,1993.7850,0.002923,1,0.071488
2,100004,100004_2002,2800,15705.0,2002.0,ALTANA AG,611.822,467.647,316.810,1550.167,...,435.506,163.8099,0.473912,0.066934,0.051152,0.034418,2625.9450,0.317065,1,-0.011356
3,100004,100004_2003,2800,16070.0,2003.0,ALTANA AG,730.220,621.432,401.348,2001.632,...,912.214,274.9449,0.499691,0.174433,0.055237,0.030364,3291.2260,0.253349,1,-0.003054
4,100004,100004_2004,2800,16436.0,2004.0,ALTANA AG,785.376,754.784,444.794,2171.847,...,847.868,337.8989,0.502377,0.103872,0.038973,0.012697,3964.8520,0.204673,1,-0.001623


In [215]:
f1_tb.columns

Index(['gvkey', 'map_key', 'SIC', 'datadate', 'fyear', 'conm', 'che', 'rect',
       'invt', 'act', 'lct', 'at', 'ppent', 'dltt', 'sale', 'ib', 'emp',
       'ivao', 'dlc', 'sstk', 'dltis', 'pstk', 'lt', 'ivst', 'avg_at', 'wc',
       'nco', 'fin', 'soft_asset', 'rsst_acc', 'ch_rec', 'ch_inv', 'cs',
       'ch_cs', 'issue', 'ch_roa'],
      dtype='object')

In [24]:
f1_tb_1 = f1_tb.loc[:,['gvkey','map_key','SIC','fyear','conm','rsst_acc','ch_rec','ch_inv','soft_asset','ch_cs','ch_roa','issue']]

In [251]:
#score_temp = scores_1.loc[:,['cik','y2','s1','window','map_key','label','label_lag']]

In [252]:
#score_temp = score_temp[score_temp.window == 1]

In [253]:
#score_temp.head()

,cik,y2,s1,window,map_key,label,label_lag
0,2135,2002,0.752746,1,30697_2001,1,1
1,2135,2003,0.544492,2,30697_2001,1,1
2,2135,2004,0.484842,3,30697_2001,1,1
3,2135,2003,0.689329,1,30697_2002,1,1
4,2135,2004,0.621834,2,30697_2002,1,1


In [342]:
df_merge = pd.merge(scores_1, f1_tb_1, on = 'map_key', how ='left')

In [343]:
df_merge.shape
score_temp.shape

(11390, 28)

(11308, 7)

In [344]:
df_merge

,cik,y1,y2,s1,s2,s3,s4,l1,l2,window,...,SIC,fyear,conm,rsst_acc,ch_rec,ch_inv,soft_asset,ch_cs,ch_roa,issue
0,2135,2001,2002,0.752746,0.752746,0.769208,0.769208,34,91,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2135,2001,2003,0.544492,0.544492,0.566316,0.566316,34,107,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2135,2001,2004,0.484842,0.484842,0.513163,0.513163,34,128,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2135,2002,2003,0.689329,0.689329,0.704758,0.704758,91,107,1,...,7374.0,2002.0,AFFILIATED COMPUTER SERVICES,1.104736,0.382524,0.001899,0.874060,0.938951,0.048537,1.0
4,2135,2002,2004,0.621834,0.621834,0.646809,0.646809,91,128,2,...,7374.0,2002.0,AFFILIATED COMPUTER SERVICES,1.104736,0.382524,0.001899,0.874060,0.938951,0.048537,1.0
5,2135,2002,2005,0.561204,0.561204,0.571958,0.571958,91,171,3,...,7374.0,2002.0,AFFILIATED COMPUTER SERVICES,1.104736,0.382524,0.001899,0.874060,0.938951,0.048537,1.0
6,2135,2003,2004,0.764921,0.764921,0.781340,0.781340,107,128,1,...,7374.0,2003.0,AFFILIATED COMPUTER SERVICES,0.089102,0.027880,-0.000984,0.856874,0.557009,-0.040117,1.0
7,2135,2003,2005,0.657212,0.657212,0.674718,0.674718,107,171,2,...,7374.0,2003.0,AFFILIATED COMPUTER SERVICES,0.089102,0.027880,-0.000984,0.856874,0.557009,-0.040117,1.0
8,2135,2003,2006,0.468065,0.468065,0.479704,0.479704,107,299,3,...,7374.0,2003.0,AFFILIATED COMPUTER SERVICES,0.089102,0.027880,-0.000984,0.856874,0.557009,-0.040117,1.0
9,2135,2004,2005,0.773985,0.773985,0.809491,0.809491,128,171,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
#check funda raw data
funda = pd.read_csv('/home/jujun/temp_10k/fscore_data/combine_funda.csv')

In [258]:
funda[funda.gvkey == 2135]

,Unnamed: 0,map_key,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,...,adjex_f,rank,au,auop,auopic,LINKPRIM,LINKTYPE,LPERMCO,LPERMNO,SIC
93689,93689,2135_1995,2135,13148.0,1995.0,INDL,C,D,SUMM_STD,PQE1,...,1.0,1.0,6.0,1.0,NaN,P,LU,23135.0,22322.0,3579
93690,93690,2135_1996,2135,13514.0,1996.0,INDL,C,D,SUMM_STD,PQE1,...,NaN,1.0,6.0,1.0,NaN,P,LU,23135.0,22322.0,3579


### prepare for reg

In [352]:
from scipy.stats import mstats

In [411]:
import math

In [345]:
df_merge_1 = df_merge.dropna()

In [377]:
df_merge_2 = df_merge_1[df_merge_1.sen_num >= 8]

In [378]:
df_merge_2.columns

Index(['cik', 'y1', 'y2', 's1', 's2', 's3', 's4', 'l1', 'l2', 'window',
       'gvkey_x', 'label', 'label_lag', 'year', 'sen_num', 'word_num',
       'map_key', 'gvkey_y', 'SIC', 'fyear', 'conm', 'rsst_acc', 'ch_rec',
       'ch_inv', 'soft_asset', 'ch_cs', 'ch_roa', 'issue'],
      dtype='object')

In [379]:
df_merge_2.isnull().sum()

cik           0
y1            0
y2            0
s1            0
s2            0
s3            0
s4            0
l1            0
l2            0
window        0
gvkey_x       0
label         0
label_lag     0
year          0
sen_num       0
word_num      0
map_key       0
gvkey_y       0
SIC           0
fyear         0
conm          0
rsst_acc      0
ch_rec        0
ch_inv        0
soft_asset    0
ch_cs         0
ch_roa        0
issue         0
dtype: int64

In [391]:
df_merge_test = df_merge_2.copy()

In [392]:
df_merge_test[['rsst_acc','ch_rec','ch_inv',
                 'soft_asset','ch_cs','ch_roa']].describe()

,rsst_acc,ch_rec,ch_inv,soft_asset,ch_cs,ch_roa
count,7289.000000,7289.000000,7289.000000,7289.000000,7289.000000,7289.000000
mean,0.481040,-0.010456,-0.008500,0.610668,358.678940,-0.014445
std,28.218354,0.223454,0.176530,0.224385,17428.944105,1.862529
min,-68.915313,-4.070211,-3.807424,-1.000000,-8.872340,-55.813320
25%,-0.071820,-0.017446,-0.006219,0.463947,-0.087012,-0.051669
50%,0.021120,0.005487,0.000000,0.648157,0.050655,0.000318
75%,0.105713,0.038288,0.018371,0.781944,0.217500,0.041519
max,1387.843137,1.129488,0.865672,0.999947,859190.000000,34.685335


In [393]:
def apply_winsorize(s):
    return mstats.winsorize(s, limits=[0.05, 0.05])

In [394]:
df_merge_test[['rsst_acc','ch_rec','ch_inv',
                 'soft_asset','ch_cs','ch_roa']] = df_merge_test[['rsst_acc',
                                                                    'ch_rec','ch_inv','soft_asset','ch_cs','ch_roa']].apply(apply_winsorize)

In [395]:
df_merge_test[['rsst_acc','ch_rec','ch_inv',
                 'soft_asset','ch_cs','ch_roa']].describe()

,rsst_acc,ch_rec,ch_inv,soft_asset,ch_cs,ch_roa
count,7289.000000,7289.000000,7289.000000,7289.000000,7289.000000,7289.000000
mean,0.013188,0.006429,0.002931,0.613033,0.087177,-0.005217
std,0.223491,0.067405,0.051790,0.212675,0.339133,0.145947
min,-0.519777,-0.162983,-0.136060,0.191035,-0.528720,-0.360910
25%,-0.071820,-0.017446,-0.006219,0.463947,-0.087012,-0.051669
50%,0.021120,0.005487,0.000000,0.648157,0.050655,0.000318
75%,0.105713,0.038288,0.018371,0.781944,0.217500,0.041519
max,0.501410,0.144468,0.114932,0.916360,0.960563,0.320476


In [396]:
df_merge_test[['sen_num','word_num']].describe()

,sen_num,word_num
count,7289.000000,7289.000000
mean,163.669914,10138.210043
std,159.860502,7526.606426
min,11.000000,632.000000
25%,71.000000,5125.000000
50%,127.000000,8644.000000
75%,203.000000,13024.000000
max,1631.000000,67639.000000


In [397]:
df_merge_test[['sen_num','word_num']] = np.log(df_merge_test[['sen_num','word_num']])

In [398]:
df_merge_test[['rsst_acc','ch_rec','ch_inv',
                 'soft_asset','ch_cs','ch_roa','sen_num','word_num']].describe()

,rsst_acc,ch_rec,ch_inv,soft_asset,ch_cs,ch_roa,sen_num,word_num
count,7289.000000,7289.000000,7289.000000,7289.000000,7289.000000,7289.000000,7289.000000,7289.000000
mean,0.013188,0.006429,0.002931,0.613033,0.087177,-0.005217,4.800609,8.992074
std,0.223491,0.067405,0.051790,0.212675,0.339133,0.145947,0.760063,0.702122
min,-0.519777,-0.162983,-0.136060,0.191035,-0.528720,-0.360910,2.397895,6.448889
25%,-0.071820,-0.017446,-0.006219,0.463947,-0.087012,-0.051669,4.262680,8.541886
50%,0.021120,0.005487,0.000000,0.648157,0.050655,0.000318,4.844187,9.064621
75%,0.105713,0.038288,0.018371,0.781944,0.217500,0.041519,5.313206,9.474549
max,0.501410,0.144468,0.114932,0.916360,0.960563,0.320476,7.396949,11.121940


In [431]:
def reg_logit_winsorize(win = None, s = None):
    if s == None:
        columns = ['rsst_acc','ch_rec','ch_inv','soft_asset','ch_cs','ch_roa','issue','sen_num', 'word_num']
    else:
        columns = ['rsst_acc','ch_rec','ch_inv','soft_asset','ch_cs','ch_roa','issue','sen_num', 'word_num'] + [s]
    if win == None:
        df_merge_temp = df_merge_2
    else:
        df_merge_temp = df_merge_1[df_merge_1.window == win]
    df_merge_target = df_merge_temp.loc[:,columns +['label']]
    print('before scale')
    print(df_merge_target.describe())
    
    
#     fig = plt.figure(figsize = (15,20))
#     ax = fig.gca()
#     df_merge_target.hist(ax = ax)

    df_merge_target[['rsst_acc','ch_rec','ch_inv',
                     'soft_asset','ch_cs','ch_roa']] = df_merge_target[['rsst_acc',
                                                                        'ch_rec','ch_inv','soft_asset','ch_cs','ch_roa']].apply(apply_winsorize)
    df_merge_target[['sen_num','word_num']] = np.log(df_merge_target[['sen_num','word_num']])
#     df_merge_target['sen_num'] = df_merge_target['sen_num'].apply(lambda x: math.log(x))
#     df_merge_target['word_num'] = df_merge_target['word_num'].apply(lambda x: math.log(x))
#    df_merge_target = df_merge_target.apply(lambda x: math.log(x['sen_num'], x['word_num']), axis=1)
    
    print('\t')
    print('after scale')
    print(df_merge_target.describe())
    print('\t')
    print(df_merge_target.label.value_counts())
    
    
    X = df_merge_target.loc[:,columns]
    X = sm.add_constant(X)
    y = df_merge_target.label
    results_orgi = sm.Logit(y, X).fit()
    print(results_orgi.summary())

In [432]:
reg_logit_winsorize(win = 1, s = None)

before scale
          rsst_acc       ch_rec       ch_inv   soft_asset          ch_cs  \
count  2723.000000  2723.000000  2723.000000  2723.000000    2723.000000   
mean      0.413662    -0.009860    -0.008159     0.611390     319.991963   
std      26.665824     0.222343     0.178536     0.226866   16465.695762   
min     -68.915313    -4.070211    -3.807424    -1.000000      -8.872340   
25%      -0.072396    -0.017437    -0.006005     0.467162      -0.089469   
50%       0.019347     0.005014     0.000000     0.651114       0.047772   
75%       0.103962     0.038168     0.018136     0.784480       0.212412   
max    1387.843137     1.129488     0.865672     0.999947  859190.000000   

            ch_roa        issue      sen_num      word_num        label  
count  2723.000000  2723.000000  2723.000000   2723.000000  2723.000000  
mean     -0.012418     0.930224   157.762394   9794.493573     0.163790  
std       1.853706     0.254816   157.996504   7432.150151     0.370153  
min   

/home/jujun/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [425]:
reg_logit_winsorize(win = 1, s = 's1')

before scale
          rsst_acc       ch_rec       ch_inv   soft_asset          ch_cs  \
count  2723.000000  2723.000000  2723.000000  2723.000000    2723.000000   
mean      0.413662    -0.009860    -0.008159     0.611390     319.991963   
std      26.665824     0.222343     0.178536     0.226866   16465.695762   
min     -68.915313    -4.070211    -3.807424    -1.000000      -8.872340   
25%      -0.072396    -0.017437    -0.006005     0.467162      -0.089469   
50%       0.019347     0.005014     0.000000     0.651114       0.047772   
75%       0.103962     0.038168     0.018136     0.784480       0.212412   
max    1387.843137     1.129488     0.865672     0.999947  859190.000000   

            ch_roa        issue      sen_num      word_num           s1  \
count  2723.000000  2723.000000  2723.000000   2723.000000  2723.000000   
mean     -0.012418     0.930224   157.762394   9794.493573     0.714824   
std       1.853706     0.254816   157.996504   7432.150151     0.111604   
mi

/home/jujun/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [433]:
reg_logit_winsorize(win = 1, s = 's3')

before scale
          rsst_acc       ch_rec       ch_inv   soft_asset          ch_cs  \
count  2723.000000  2723.000000  2723.000000  2723.000000    2723.000000   
mean      0.413662    -0.009860    -0.008159     0.611390     319.991963   
std      26.665824     0.222343     0.178536     0.226866   16465.695762   
min     -68.915313    -4.070211    -3.807424    -1.000000      -8.872340   
25%      -0.072396    -0.017437    -0.006005     0.467162      -0.089469   
50%       0.019347     0.005014     0.000000     0.651114       0.047772   
75%       0.103962     0.038168     0.018136     0.784480       0.212412   
max    1387.843137     1.129488     0.865672     0.999947  859190.000000   

            ch_roa        issue      sen_num      word_num           s3  \
count  2723.000000  2723.000000  2723.000000   2723.000000  2723.000000   
mean     -0.012418     0.930224   157.762394   9794.493573     0.725237   
std       1.853706     0.254816   157.996504   7432.150151     0.112008   
mi

/home/jujun/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Optimization terminated successfully.
         Current function value: 0.418534
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                 2723
Model:                          Logit   Df Residuals:                     2712
Method:                           MLE   Df Model:                           10
Date:                Sun, 21 Jun 2020   Pseudo R-squ.:                 0.06138
Time:                        15:53:38   Log-Likelihood:                -1139.7
converged:                       True   LL-Null:                       -1214.2
                                        LLR p-value:                 5.861e-27
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.1119      1.236     -3.326      0.001      -6.535      -1.689
rsst_acc       0.8703      0.

In [278]:
def reg(win = None, s = None,):
    columns = ['rsst_acc','ch_rec','ch_inv','soft_asset','ch_cs','ch_roa','issue'] + [s]
    if win == None:
        df_merge_temp = df_merge_1
    else:
        df_merge_temp = df_merge_1[df_merge_1.window == win]
    df_merge_target = df_merge_temp.loc[:,columns +['label']]
    print('before scale')
    print(df_merge_target.describe())
    
    scaler = MinMaxScaler()
    df_merge_target[['rsst_acc','ch_rec','ch_inv',
                     'soft_asset','ch_cs','ch_roa']] = scaler.fit_transform(df_merge_target[['rsst_acc',
                                                                                    'ch_rec','ch_inv','soft_asset','ch_cs','ch_roa']])
    print('\t')
    print('before scale')
    print(df_merge_target.describe())
    print('\t')
    print(df_merge_target.label.value_counts())
    
    
    X = df_merge_target.loc[:,columns]
    X = sm.add_constant(X)
    y = df_merge_target.label
    results_orgi = sm.OLS(y, X).fit()
    print(results_orgi.summary())

In [279]:
reg(win = 1, s = 's1')

before scale
          rsst_acc       ch_rec       ch_inv   soft_asset          ch_cs  \
count  2709.000000  2709.000000  2709.000000  2709.000000    2709.000000   
mean      0.418393    -0.008833    -0.007149     0.611738     321.646640   
std      26.734416     0.218749     0.174370     0.226786   16508.187486   
min     -68.915313    -4.070211    -3.807424    -1.000000      -8.872340   
25%      -0.072384    -0.017427    -0.005974     0.467554      -0.087600   
50%       0.019275     0.005036     0.000000     0.650530       0.048225   
75%       0.103942     0.038018     0.018097     0.785279       0.213294   
max    1387.843137     1.129488     0.865672     0.999947  859190.000000   

            ch_roa        issue           s1       label  
count  2709.000000  2709.000000  2709.000000  2709.00000  
mean     -0.012358     0.930602     0.714926     0.16316  
std       1.858482     0.254177     0.111847     0.36958  
min     -55.813320     0.000000     0.166167     0.00000  
25%    

/home/jujun/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [280]:
reg(win = 2, s = 's1')

before scale
          rsst_acc       ch_rec       ch_inv   soft_asset          ch_cs  \
count  2405.000000  2405.000000  2405.000000  2405.000000    2405.000000   
mean      0.488062    -0.011564    -0.009627     0.611075     361.878286   
std      28.364080     0.226609     0.181846     0.223662   17520.481156   
min     -68.915313    -4.070211    -3.807424    -1.000000      -8.872340   
25%      -0.072609    -0.017871    -0.006733     0.465863      -0.088172   
50%       0.020327     0.005350     0.000000     0.648819       0.050831   
75%       0.104321     0.037946     0.017864     0.781428       0.213992   
max    1387.843137     0.732089     0.817236     0.999947  859190.000000   

            ch_roa        issue           s1        label  
count  2405.000000  2405.000000  2405.000000  2405.000000  
mean     -0.019219     0.933472     0.578338     0.155925  
std       1.828933     0.249255     0.123701     0.362860  
min     -55.813320     0.000000     0.155790     0.000000  
25

/home/jujun/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [281]:
reg(win = 3, s = 's1')

before scale
          rsst_acc       ch_rec       ch_inv   soft_asset          ch_cs  \
count  2143.000000  2143.000000  2143.000000  2143.000000    2143.000000   
mean      0.562850    -0.009379    -0.007609     0.610165     407.258957   
std      30.045352     0.220866     0.168392     0.222337   18560.658393   
min     -68.915313    -4.070211    -3.538343    -1.000000      -8.872340   
25%      -0.070162    -0.017056    -0.006196     0.462781      -0.081962   
50%       0.023638     0.006306     0.000000     0.643678       0.054795   
75%       0.110587     0.039231     0.019443     0.780021       0.227517   
max    1387.843137     0.732089     0.651163     0.999947  859190.000000   

            ch_roa        issue           s1        label  
count  2143.000000  2143.000000  2143.000000  2143.000000  
mean     -0.011187     0.935138     0.507400     0.144657  
std       1.918873     0.246340     0.122197     0.351837  
min     -55.813320     0.000000     0.155743     0.000000  
25

/home/jujun/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [282]:
s

In [283]:
reg_logit(win = 1, s = 's1')

before scale
          rsst_acc       ch_rec       ch_inv   soft_asset          ch_cs  \
count  2709.000000  2709.000000  2709.000000  2709.000000    2709.000000   
mean      0.418393    -0.008833    -0.007149     0.611738     321.646640   
std      26.734416     0.218749     0.174370     0.226786   16508.187486   
min     -68.915313    -4.070211    -3.807424    -1.000000      -8.872340   
25%      -0.072384    -0.017427    -0.005974     0.467554      -0.087600   
50%       0.019275     0.005036     0.000000     0.650530       0.048225   
75%       0.103942     0.038018     0.018097     0.785279       0.213294   
max    1387.843137     1.129488     0.865672     0.999947  859190.000000   

            ch_roa        issue           s1       label  
count  2709.000000  2709.000000  2709.000000  2709.00000  
mean     -0.012358     0.930602     0.714926     0.16316  
std       1.858482     0.254177     0.111847     0.36958  
min     -55.813320     0.000000     0.166167     0.00000  
25%    

/home/jujun/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [284]:
reg_logit(win = 1, s = 's3')

before scale
          rsst_acc       ch_rec       ch_inv   soft_asset          ch_cs  \
count  2709.000000  2709.000000  2709.000000  2709.000000    2709.000000   
mean      0.418393    -0.008833    -0.007149     0.611738     321.646640   
std      26.734416     0.218749     0.174370     0.226786   16508.187486   
min     -68.915313    -4.070211    -3.807424    -1.000000      -8.872340   
25%      -0.072384    -0.017427    -0.005974     0.467554      -0.087600   
50%       0.019275     0.005036     0.000000     0.650530       0.048225   
75%       0.103942     0.038018     0.018097     0.785279       0.213294   
max    1387.843137     1.129488     0.865672     0.999947  859190.000000   

            ch_roa        issue           s3       label  
count  2709.000000  2709.000000  2709.000000  2709.00000  
mean     -0.012358     0.930602     0.725312     0.16316  
std       1.858482     0.254177     0.112259     0.36958  
min     -55.813320     0.000000     0.154513     0.00000  
25%    

/home/jujun/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                 2709
Model:                          Logit   Df Residuals:                     2700
Method:                           MLE   Df Model:                            8
Date:                Sat, 20 Jun 2020   Pseudo R-squ.:                 0.02620
Time:                        20:28:25   Log-Likelihood:                -1173.6
converged:                       True   LL-Null:                       -1205.2
                                        LLR p-value:                 1.113e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -12.2372      3.093     -3.956      0.000     -18.300      -6.174
rsst_acc      -3.4178      7.635     -0.448      0.654     -18.382      11.547
ch_rec        10.0074      2.743      3.648      0.0

In [260]:
df_merge_1.shape

(7257, 26)

In [239]:
df_merge_1.label.value_counts()

0    2267
1     442
Name: label, dtype: int64

In [240]:
df_merge_1.loc[:,['label','s1','rsst_acc','ch_rec','ch_inv','soft_asset','ch_cs','ch_roa','issue']].describe()

,label,s1,rsst_acc,ch_rec,ch_inv,soft_asset,ch_cs,ch_roa,issue
count,2709.00000,2709.000000,2709.000000,2709.000000,2709.000000,2709.000000,2709.000000,2709.000000,2709.000000
mean,0.16316,0.714926,0.418393,-0.008833,-0.007149,0.611738,321.646640,-0.012358,0.930602
std,0.36958,0.111847,26.734416,0.218749,0.174370,0.226786,16508.187486,1.858482,0.254177
min,0.00000,0.166167,-68.915313,-4.070211,-3.807424,-1.000000,-8.872340,-55.813320,0.000000
25%,0.00000,0.655505,-0.072384,-0.017427,-0.005974,0.467554,-0.087600,-0.051506,1.000000
50%,0.00000,0.729345,0.019275,0.005036,0.000000,0.650530,0.048225,0.000318,1.000000
75%,0.00000,0.791939,0.103942,0.038018,0.018097,0.785279,0.213294,0.042594,1.000000
max,1.00000,0.979765,1387.843137,1.129488,0.865672,0.999947,859190.000000,34.685335,1.000000


In [236]:
df_merge_1.loc[:,['label','s1','rsst_acc','ch_rec','ch_inv','soft_asset','ch_cs','ch_roa','issue']].describe()

,label,s1,rsst_acc,ch_rec,ch_inv,soft_asset,ch_cs,ch_roa,issue
count,2709.00000,2709.000000,2709.000000,2709.000000,2709.000000,2709.000000,2709.000000,2709.000000,2709.000000
mean,0.16316,0.714926,0.047595,-0.008833,-0.007149,0.611738,321.646640,-0.012358,0.930602
std,0.36958,0.111847,0.018352,0.218749,0.174370,0.226786,16508.187486,1.858482,0.254177
min,0.00000,0.166167,0.000000,-4.070211,-3.807424,-1.000000,-8.872340,-55.813320,0.000000
25%,0.00000,0.655505,0.047258,-0.017427,-0.005974,0.467554,-0.087600,-0.051506,1.000000
50%,0.00000,0.729345,0.047321,0.005036,0.000000,0.650530,0.048225,0.000318,1.000000
75%,0.00000,0.791939,0.047379,0.038018,0.018097,0.785279,0.213294,0.042594,1.000000
max,1.00000,0.979765,1.000000,1.129488,0.865672,0.999947,859190.000000,34.685335,1.000000


In [241]:
from sklearn.preprocessing import MinMaxScaler

In [242]:
df_merge_2 = df_merge_1.copy()
scaler = MinMaxScaler()
df_merge_2[['rsst_acc','ch_rec','ch_inv','soft_asset','ch_cs','ch_roa']] = scaler.fit_transform(df_merge_2[['rsst_acc','ch_rec','ch_inv','soft_asset','ch_cs','ch_roa']])

In [243]:
df_merge_2.loc[:,['label','s1','rsst_acc','ch_rec','ch_inv','soft_asset','ch_cs','ch_roa','issue']].describe()

,label,s1,rsst_acc,ch_rec,ch_inv,soft_asset,ch_cs,ch_roa,issue
count,2709.00000,2709.000000,2709.000000,2709.000000,2709.000000,2709.000000,2709.000000,2709.000000,2709.000000
mean,0.16316,0.714926,0.047595,0.781079,0.813224,0.805891,0.000385,0.616594,0.930602
std,0.36958,0.111847,0.018352,0.042070,0.037314,0.113396,0.019213,0.020536,0.254177
min,0.00000,0.166167,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.655505,0.047258,0.779427,0.813476,0.733796,0.000010,0.616162,1.000000
50%,0.00000,0.729345,0.047321,0.783747,0.814754,0.825287,0.000010,0.616734,1.000000
75%,0.00000,0.791939,0.047379,0.790090,0.818627,0.892663,0.000011,0.617202,1.000000
max,1.00000,0.979765,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [244]:
import statsmodels.api as sm

In [245]:
X_orgi = df_merge_2.loc[:,['rsst_acc','ch_rec','ch_inv','soft_asset','ch_cs','ch_roa','issue']]

X_orgi = sm.add_constant(X_orgi)

/home/jujun/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [246]:
X = df_merge_2.loc[:,['rsst_acc','ch_rec','ch_inv','soft_asset','ch_cs','ch_roa','issue','s1']]

In [247]:
X = sm.add_constant(X)

/home/jujun/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [248]:
y = df_merge_2.label

In [249]:
results_orgi = sm.OLS(y, X_orgi).fit()
print(results_orgi.summary())

                            OLS Regression Results                            
Dep. Variable:                  label   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     4.510
Date:                Sat, 20 Jun 2020   Prob (F-statistic):           5.21e-05
Time:                        17:03:28   Log-Likelihood:                -1131.2
No. Observations:                2709   AIC:                             2278.
Df Residuals:                    2701   BIC:                             2326.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5346      0.271     -1.975      0.0

In [250]:
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  label   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     5.877
Date:                Sat, 20 Jun 2020   Prob (F-statistic):           1.78e-07
Time:                        17:03:29   Log-Likelihood:                -1123.5
No. Observations:                2709   AIC:                             2265.
Df Residuals:                    2700   BIC:                             2318.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3950      0.272     -1.451      0.1